In [2]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt 

In [3]:
def json_get_length(path):
    with open(path, encoding="utf-8") as f:
        lines = f.readlines()
        return len(lines)

In [4]:
def json_to_df(path, l):
    with open(path, encoding="utf-8") as f:
        dic = {}
        for i in tqdm(range(l)):
            line = f.readline()
            dic_new = dict(json.loads(line))
            if i == 0:
                dic = dic_new
                for key in dic:
                    dic[key] = [dic[key]]
            else:
                for key in dic:
                    dic[key].append(dic_new[key])
        f.close()
        dic = pd.DataFrame(dic)
        return dic

In [4]:
#read review file
l_review=json_get_length('review.json')
review = json_to_df('review.json',l_review)

100%|██████████| 6685900/6685900 [01:39<00:00, 67325.47it/s]


In [6]:
#read tip file
l_tip=json_get_length('tip.json')
tip = json_to_df('tip.json',l_tip)

100%|██████████| 1223094/1223094 [00:09<00:00, 125031.04it/s]


In [10]:
unique_business_id = set(list(review['business_id']))

In [11]:
unique_user_id = set(list(review['user_id']))

In [13]:
#find tips only for these business
tip = tip[tip['business_id'].isin(unique_business_id)]

In [5]:
#read business file
l_business=json_get_length('business.json')
business = json_to_df('business.json',l_business)

FileNotFoundError: [Errno 2] No such file or directory: 'business.json'

In [15]:
#find information only for these business
business = business[business['business_id'].isin(unique_business_id)]

In [16]:
#read user file
l_user=json_get_length('user.json')
user = json_to_df('user.json',l_user)


 58%|█████▊    | 945961/1637138 [00:28<00:18, 37292.21it/s]


100%|██████████| 1637138/1637138 [00:46<00:00, 34952.26it/s]


In [20]:
#find users only those who write reviews in review files
user = user[user['user_id'].isin(unique_user_id)]

In [21]:
len(user)

1637138

In [32]:
business_categories=business['categories']
business_categories

0                                         Golf, Active Life
1         Specialty Food, Restaurants, Dim Sum, Imported...
2                         Sushi Bars, Restaurants, Japanese
3                             Insurance, Financial Services
4         Plumbing, Shopping, Local Services, Home Servi...
                                ...                        
192604    Water Purification Services, Water Heater Inst...
192605                 Books, Mags, Music & Video, Shopping
192606    Home Services, Contractors, Landscaping, Mason...
192607                               Beauty & Spas, Barbers
192608                               Massage, Beauty & Spas
Name: categories, Length: 192606, dtype: object

In [34]:
business_categories=business['categories']
categories = set()
count = 0
for business_category in business_categories:
    if not business_category:
        print(count)
    count += 1

265
597
1020
1332
1770
2399
2818
2909
2919
2945
2967
3552
3685
4244
4626
4789
5288
5450
6496
7611
7726
8306
8347
8355
8395
8699
8782
8834
9169
10716
11082
11148
11476
11910
12420
13777
14169
14368
14456
14460
14855
16106
16868
17893
17992
17998
19159
19244
19271
19461
20605
21148
21470
22726
23091
23303
24166
24568
24714
27350
27911
28623
29692
30378
30719
31443
31735
32061
32158
32874
33419
33786
34541
34542
35039
35121
35217
35648
37535
38197
38359
38413
38578
39327
40559
40840
40948
41340
41462
41783
43655
44035
44148
44285
44292
44298
44625
44673
44838
45055
45136
45670
46322
46435
46615
46711
46878
47035
47040
47580
48372
48673
49017
49125
49355
49494
49818
50074
50752
50790
50867
51500
51807
51838
53248
53375
53477
53822
53902
54188
54238
54756
55034
55274
55579
56356
56635
56841
57739
58400
58708
59104
59206
59508
60166
61214
61895
62040
64207
64554
64629
64981
65280
66116
66258
66450
66746
67323
67483
67744
67825
68138
68465
68508
69021
69265
70508
70713
71327
71532
71924
72121

In [33]:
business_categories=' '.join(business['categories'])

categories=pd.DataFrame(business_categories.split(','),columns=['category'])
counts=categories.category.value_counts()
print("There are ",len(counts)," different categories of Businesses in Yelp!")
#prep for chart
counts=counts.sort_values(ascending=False)
counts=counts.iloc[0:20]

#chart
plt.figure(figsize=(16,5))
ax = sns.barplot(counts.index, counts.values, alpha=0.8)
plt.title("Top Categories in Yelp Business",fontsize=20)
locs, labels = plt.xticks()
plt.setp(labels, rotation=80)
plt.ylabel('Number of Businesses', fontsize=16)
plt.xlabel('Categories', fontsize=16)

#adding the text labels
rects = ax.patches
labels = counts.values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')

plt.show()

TypeError: sequence item 265: expected str instance, NoneType found

In [27]:
business.loc[1, 'categories']

'Specialty Food, Restaurants, Dim Sum, Imported Food, Food, Chinese, Ethnic Food, Seafood'

In [35]:
business.loc[265, ]

business_id     xOR85RicYj642O3_iJ7hgg
name                Phoenix Valuations
address               6340 E Thomas Rd
city                        Scottsdale
state                               AZ
postal_code                      85251
latitude                       33.4804
longitude                     -111.946
stars                              3.5
review_count                         3
is_open                              1
attributes                        None
categories                        None
hours                             None
Name: 265, dtype: object

In [39]:
review.loc[269914, 'text']

'Andrew is a class act.  You can tell he has been involved for many years with all kinds of different appraisals from luxury homes to complex property. He has serious commitment to making sure the data is accurate.  On top of that, he is a good guy.'

In [72]:
business.index = range(len(business))
restaurant = []
categories_count = {}
for i in tqdm(range(len(business))):
    if business.loc[i, 'categories']:
        if "Restaurants" in business.loc[i, 'categories']:
            restaurant.append(i)
            curr = business.loc[i, 'categories'].split(',')
            for e in curr:
                categories_count[e] = categories_count.get(e, 0)+1


100%|██████████| 192606/192606 [00:06<00:00, 30026.40it/s]


In [76]:
sorted_cat

{'Specialty Food': 300,
 ' Restaurants': 41423,
 ' Dim Sum': 225,
 ' Imported Food': 217,
 ' Food': 11956,
 ' Chinese': 2867,
 ' Ethnic Food': 755,
 ' Seafood': 1916,
 'Sushi Bars': 651,
 ' Japanese': 1898,
 'Restaurants': 17948,
 ' Breakfast & Brunch': 4231,
 ' Mexican': 3033,
 ' Tacos': 210,
 ' Tex-Mex': 570,
 ' Fast Food': 5320,
 'Italian': 1335,
 ' Pizza': 4658,
 ' Chicken Wings': 2115,
 ' Italian': 3381,
 'Sandwiches': 1808,
 ' Salad': 2061,
 ' Burgers': 4138,
 ' Comfort Food': 704,
 'Mexican': 1585,
 ' Patisserie/Cake Shop': 76,
 ' Bars': 6888,
 ' Nightlife': 7133,
 'Juice Bars & Smoothies': 161,
 ' Vegan': 686,
 ' Tapas Bars': 339,
 ' Southern': 424,
 ' Coffee & Tea': 3041,
 ' American (Traditional)': 5542,
 ' American (New)': 3790,
 ' Steakhouses': 1206,
 'Chinese': 1561,
 ' Modern European': 171,
 ' Wine Bars': 842,
 'Coffee & Tea': 606,
 ' Hookah Bars': 129,
 ' Persian/Iranian': 140,
 ' Country Dance Halls': 11,
 ' Middle Eastern': 972,
 ' Beer': 1004,
 ' Wine & Spirits': 113

In [74]:
business.iloc[8019, ]

business_id                                FXX8ge8ZM5JkEpL0Il4O0Q
name                                                  Kings Glass
address                                     6230 Centre Street SE
city                                                      Calgary
state                                                          AB
postal_code                                               T2H 0C6
latitude                                                  50.9955
longitude                                                -114.064
stars                                                         3.5
review_count                                                    3
is_open                                                         1
attributes                                                   None
categories                        Automotive, Auto Glass Services
hours           {'Monday': '8:30-17:0', 'Tuesday': '8:30-17:0'...
Name: 8019, dtype: object

In [47]:
dt=np.dtype(business['categories'])

In [79]:
import operator

In [82]:
sorted_cat = sorted(categories_count.items(),key = operator.itemgetter(1))

In [84]:
sorted_cat[::-1]

[(' Restaurants', 41423),
 ('Restaurants', 17948),
 (' Food', 11956),
 (' Nightlife', 7133),
 (' Bars', 6888),
 (' American (Traditional)', 5542),
 (' Sandwiches', 5524),
 (' Fast Food', 5320),
 (' Pizza', 4658),
 (' Breakfast & Brunch', 4231),
 (' Burgers', 4138),
 (' American (New)', 3790),
 (' Italian', 3381),
 (' Coffee & Tea', 3041),
 (' Mexican', 3033),
 (' Chinese', 2867),
 ('Food', 2844),
 (' Cafes', 2561),
 (' Event Planning & Services', 2277),
 ('Pizza', 2146),
 (' Chicken Wings', 2115),
 (' Salad', 2061),
 ('Fast Food', 1937),
 (' Seafood', 1916),
 (' Japanese', 1898),
 ('Sandwiches', 1808),
 (' Specialty Food', 1791),
 (' Sushi Bars', 1607),
 ('Mexican', 1585),
 ('American (Traditional)', 1565),
 ('Chinese', 1561),
 (' Caterers', 1544),
 (' Canadian (New)', 1518),
 (' Sports Bars', 1511),
 (' Delis', 1502),
 (' Asian Fusion', 1499),
 (' Bakeries', 1498),
 (' Desserts', 1442),
 ('Nightlife', 1429),
 (' Mediterranean', 1399),
 ('Italian', 1335),
 ('Bars', 1294),
 (' Barbeque'

In [1]:
business

NameError: name 'business' is not defined